In [ ]:
import pdal
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.image
import pathlib
import subprocess
import json
import tqdm.auto as tqdm
import shapely.geometry
import numpy as np
from pycocotools.coco import COCO
import skimage.io
import open3d
from open3d.web_visualizer import draw

import numpy as np
import cv2
import zlib
import base64
import io
from PIL import Image

print(f"Using pdal version {pdal.__version__}")

In [ ]:
def make_las_bounds(lasfolder):
    """ """
    lasfiles = [p for p in pathlib.Path(lasfolder).iterdir() if p.suffix == ".laz"]
    df_las = []
    for p in tqdm.tqdm(lasfiles):
        # decode stdout from bytestring and convert to a dictionary
        result = subprocess.run(
            ["pdal", "info", str(p)], stderr=subprocess.PIPE, stdout=subprocess.PIPE
        )
        json_result = json.loads(result.stdout.decode())
        maxx, maxy, maxz, minx, miny, minz = json_result["stats"]["bbox"]["native"][
            "bbox"
        ].values()
        geom = shapely.geometry.box(minx, miny, maxx, maxy)
        df_las.append((p.stem, str(p), maxx, maxy, maxz, minx, miny, minz, geom))
    df_las = pd.DataFrame(
        df_las,
        columns=[
            "name",
            "path",
            "maxx",
            "maxy",
            "maxz",
            "minx",
            "miny",
            "minz",
            "geometry",
        ],
    )
    df_las = gpd.GeoDataFrame(df_las, geometry="geometry", crs="epsg:28992")
    return df_las


def find_tile_bounds(root_tilepath, concat=True):
    cell_files = []
    for p in pathlib.Path(root_tilepath).iterdir():
        if p.is_dir():
            cell_files += find_tile_bounds(p, concat=False)
        elif p.is_file() and p.suffix == ".gpkg" and "cells_intersect" in p.stem:
            df = gpd.read_file(p)
            df["name"] = p.stem.replace("_cells_intersects", "")
            cell_files.append(df.copy())
    if concat:
        cell_files = pd.concat(cell_files)
    return cell_files


def get_lasdata(las_file, xmin, xmax, ymin, ymax):
    jsondata = f"""
    [
        "{lasfile.path}",
        {{
            "type":"filters.expression",
            "expression":"(X >= {xmin} && X <= {xmax} && Y >= {ymin} && Y <= {ymax})"
        }}
    ]
    """
    pipeline = pdal.Pipeline(jsondata)
    count = pipeline.execute()
    arrays = pipeline.arrays
    assert len(arrays) == 1
    #         metadata = pipeline.metadata
    #         log = pipeline.log
    arrays = pd.DataFrame(arrays[0])
    return arrays


def get_lasground(las_file, xmin, xmax, ymin, ymax):
    jsondata = f"""
    [
        "{lasfile.path}",
        {{
            "type":"filters.expression",
            "expression":"(X >= {xmin} && X <= {xmax} && Y >= {ymin} && Y <= {ymax})"
        }},
        {{
            "type":"filters.assign",
            "assignment":"Classification[:]=0"
        }},
        {{
            "type":"filters.smrf",
            "scalar": 1.2,
            "slope": 0.2,
            "threshold": 0.45,
            "window": 16.0
        }}
    ]
    """
    pipeline = pdal.Pipeline(jsondata)
    count = pipeline.execute()
    arrays = pipeline.arrays
    assert len(arrays) == 1
    #         metadata = pipeline.metadata
    #         log = pipeline.log
    arrays = pd.DataFrame(arrays[0])
    return arrays


def base64_2_mask(s):
    z = zlib.decompress(base64.b64decode(s))
    n = np.frombuffer(z, np.uint8)
    mask = cv2.imdecode(n, cv2.IMREAD_UNCHANGED)[:, :, 3].astype(bool)
    return mask


def mask_2_base64(mask):
    img_pil = Image.fromarray(np.array(mask, dtype=np.uint8))
    img_pil.putpalette([0, 0, 0, 255, 255, 255])
    bytes_io = io.BytesIO()
    img_pil.save(bytes_io, format="PNG", transparency=0, optimize=0)
    bytes = bytes_io.getvalue()
    return base64.b64encode(zlib.compress(bytes)).decode("utf-8")


def get_overlap_laz(gdf_laz, gdf_tiles):
    """ """
    # Create a spatial index for gdf2 to efficiently query overlaps
    sindex_gdf_tiles = gdf_tiles.sindex
    # Use a list comprehension with any() to find overlaps using the spatial index
    overlaps_mask = [
        any(
            gdf_tiles.iloc[
                sindex_gdf_tiles.query(geometry, predicate="intersects")
            ].geometry.intersects(geometry)
        )
        for geometry in gdf_laz.geometry
    ]
    # Apply the mask to gdf1 to filter rows that overlap with gdf2
    overlapping_gdf = gdf_laz[overlaps_mask]
    return overlapping_gdf


def zip_lasfiles(gdf_las):
    # List of file paths you want to include in the zip file
    files_to_zip = list(gdf_las.path)
    zip_file_name = "lasfiles.zip"
    # Create a zip file
    with zipfile.ZipFile(zip_file_name, "w", zipfile.ZIP_DEFLATED) as zipf:
        for file in files_to_zip:
            # Add each file to the zip file
            zipf.write(file, arcname=file.split("/")[-1])
    print(f"Created zip file {zip_file_name} containing {len(files_to_zip)} files.")


def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x["area"]), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones(
        (
            sorted_anns[0]["segmentation"].shape[0],
            sorted_anns[0]["segmentation"].shape[1],
            4,
        )
    )
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann["segmentation"]
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
# config
path_lasdata = pathlib.Path("data")
path_lasbounds = pathlib.Path("las_bounds_selected.gpkg")
path_tifftiles = pathlib.Path("data/tile_dataset")
debug = True
# path with geometry of las files (bounding box)
if not path_lasbounds.exists():
    gdf_las = make_las_bounds(path_lasdata)
    gdf_las.to_file(path_lasbounds)
else:
    print(f"File: '{path_lasbounds}' already exists!")

# geodataframe with geometries of las files
gdf_las = gpd.read_file(path_lasbounds)
print(f"Found {len(gdf_las)} las files")

# geodataframe with geometries of tiles
gdf_tilebounds = find_tile_bounds(path_tifftiles)
gdf_tilebounds_selected = gdf_tilebounds.sjoin(gdf_las, how="inner")

if debug:
    fig, ax = plt.subplots(1, 1, figsize=(12, 5))
    gdf_las.plot(ax=ax, edgecolor="black", facecolor="None", linewidth=1)
    gdf_tilebounds_selected.plot(ax=ax)

In [ ]:
idx = 0
overlapping_gdf = get_overlap_laz(gdf_las, gdf_tilebounds_selected.iloc[idx : idx + 1])
overlapping_gdf

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 12))
gdf_las.plot(ax=ax, facecolor="white", edgecolor="black")
overlapping_gdf.plot(ax=ax, facecolor="white", edgecolor="red")
gdf_tilebounds_selected.plot(ax=ax, facecolor="white", edgecolor="blue")
gdf_tilebounds_selected.iloc[idx : idx + 1].plot(
    ax=ax, facecolor="white", edgecolor="green"
)

In [ ]:
###### aanpak ######

# 1. per tile, haal laz data op en sla op als copc laz, voeg bestandsnaam toe aan tekstbestand
# 2. maak een vpc file van alle bestanden die in het tekstbestand staan


def tile_to_copc(gdf_laz, xmin, xmax, ymin, ymax):
    """
    Gets LiDAR point cloud data from file, selects data within tile and exports as Cloud Optimized Point Cloud format
    """
    abs_fp_out = "test.copc"
    jsondata = f"""
    [
        {gdf_laz.iloc[0].path},
        {"type":"filters.expression"},
        {"expression":"(X >= {xmin} && X <= {xmax} && Y >= {ymin} && Y <= {ymax})"}, 
        {"type": "writers.copc", filename': {abs_fp_out}}
        
    ]"""
    print(jsondata)
    pipeline = pdal.Pipeline(jsondata)
    count = pipeline.execute()
    return


tile_to_copc(
    overlapping_gdf,
    gdf_tilebounds_selected.iloc[idx]["xmin"],
    gdf_tilebounds_selected.iloc[idx]["xmax"],
    gdf_tilebounds_selected.iloc[idx]["ymin"],
    gdf_tilebounds_selected.iloc[idx]["ymax"],
)

get_lasdata(las_file, xmin, xmax, ymin, ymax)

In [ ]:
idx = 47
overlapping_gdf = get_overlap_laz(gdf_las, gdf_tilebounds_selected.iloc[idx : idx + 1])
fig, ax = plt.subplots(1, 1, figsize=(25, 20))
gdf_las.plot(ax=ax, facecolor="white", edgecolor="black")
overlapping_gdf.plot(ax=ax, facecolor="white", edgecolor="red")
gdf_tilebounds_selected.plot(ax=ax, facecolor="white", edgecolor="blue")
gdf_tilebounds_selected.iloc[idx : idx + 1].plot(
    ax=ax, facecolor="white", edgecolor="green"
)

In [ ]:
dataDir = pathlib.Path(
    "276114_20230714 MUG Hoorn Enkhuizen orthomosaic deel 1/2023-12-11 10_00_58.048"
)
dataDir = pathlib.Path(
    "data/supervisely/277426_20230714 MUG Hoorn Enkhuizen orthomosaic deel 6/2023-12-11 12_52_14.810"
)
annFile = dataDir.joinpath("annotations/instances.json")
imgDir = dataDir.joinpath("images")

# initialize COCO api for instance annotations
coco = COCO(annFile)
print("")

cats = coco.loadCats(coco.getCatIds())
nms = [cat["name"] for cat in cats]
print("COCO categories: \n{}\n".format(" ".join(nms)))

# nms = set([cat["supercategory"] for cat in cats])
# print("COCO supercategories: \n{}".format(" ".join(nms)))

catIds = coco.getCatIds(catNms=["twijfel opschot", "opschot"])
# imgIds = coco.getImgIds(catIds=catIds)
imgIds = coco.getImgIds()

plt.close("all")
for i, imgId in enumerate(imgIds):
    # Find tile bounds (X, Y) based on name
    imgName = ".".join(coco.imgs[imgId]["file_name"].split(".")[:-1])
    tile1a = imgName.split("_")[0]
    tile1b = int(imgName.split("_")[-1])
    cellfile = gdf_tilebounds[
        (gdf_tilebounds.index == tile1b) & (gdf_tilebounds.name == tile1a)
    ].copy()
    assert len(cellfile) == 1
    cellfile = cellfile.iloc[0, :].copy()
    print(f"  - {i+1:>02d}/{len(imgIds):>02d} rgb tile: '{tile1a}' ('{tile1b}')")

    # COCO format img en annotation
    img = coco.loadImgs(imgId)[0]

    # Laad image in
    imgPath = imgDir.joinpath(img["file_name"])
    imgRGB = skimage.io.imread(imgPath)
    imgGray = skimage.io.imread(imgPath, as_gray=True)

    # Laad annotation in
    annIds = coco.getAnnIds(imgIds=img["id"], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)

    # Read LIDAR data within the bounds of the image
    matching_lasfiles = gdf_las[gdf_las.intersects(cellfile.geometry)].copy()
    lidardata1 = []
    lidardata2 = []
    for lasfile in matching_lasfiles.itertuples():
        print(f"    - las file: {lasfile.path}")
        array1 = get_lasdata(
            lasfile.path, cellfile.xmin, cellfile.xmax, cellfile.ymin, cellfile.ymax
        )
        array1["lasfile"] = pathlib.Path(lasfile.path).stem
        array2 = get_lasground(
            lasfile.path, cellfile.xmin, cellfile.xmax, cellfile.ymin, cellfile.ymax
        )
        array2["lasfile"] = pathlib.Path(lasfile.path).stem
        lidardata1.append(array1)
        lidardata2.append(array2)
    lidardata1 = pd.concat(lidardata1, ignore_index=True)
    lidardata2 = pd.concat(lidardata2, ignore_index=True)
    xstep = (cellfile.xmax - cellfile.xmin) / imgRGB.shape[1]
    ystep = (cellfile.ymax - cellfile.ymin) / imgRGB.shape[0]
    lidardata1["m"] = ((cellfile.ymax - lidardata1.Y) / ystep).astype(int)
    lidardata1["n"] = ((lidardata1.X - cellfile.xmin) / xstep).astype(int)
    lidardata2["m"] = ((cellfile.ymax - lidardata2.Y) / ystep).astype(int)
    lidardata2["n"] = ((lidardata2.X - cellfile.xmin) / xstep).astype(int)

    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(lidardata1[["X", "Y", "Z"]].to_numpy())
    plane_model, inliers = pcd.segment_plane(
        distance_threshold=0.01, ransac_n=3, num_iterations=1000
    )
    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)

    lidardata3 = pd.DataFrame(np.asarray(outlier_cloud.points), columns=["X", "Y", "Z"])
    lidardata3["m"] = ((cellfile.ymax - lidardata3.Y) / ystep).astype(int)
    lidardata3["n"] = ((lidardata3.X - cellfile.xmin) / xstep).astype(int)

    # Make plot
    fig, axs = plt.subplots(figsize=(15, 5), ncols=3)
    plt.sca(axs[0])
    plt.imshow(imgGray, cmap="gray")
    coco.showAnns(anns, draw_bbox=True)

    lp1 = gpd.GeoDataFrame(
        lidardata1, geometry=gpd.GeoSeries.from_xy(lidardata1.n, lidardata1.m)
    )
    lp2 = gpd.GeoDataFrame(
        lidardata2, geometry=gpd.GeoSeries.from_xy(lidardata2.n, lidardata2.m)
    )
    lp3 = gpd.GeoDataFrame(
        lidardata3, geometry=gpd.GeoSeries.from_xy(lidardata3.n, lidardata3.m)
    )
    #     lp1.plot(ax=axs[1], column="Z")
    lp1[(lp1.NumberOfReturns > 1) & (lp1.ReturnNumber == 1)].plot(ax=axs[1], column="Z")
    #     lp2[lp2.Classification != 2].plot(ax=axs[2], column="Z")
    lp3.plot(ax=axs[2], column="Z")

    for ax in axs[1:]:
        ax.invert_yaxis()
        ax.set_xlim(axs[0].get_xlim())
        ax.set_ylim(axs[0].get_ylim())
    for ax in axs:
        ax.axis("off")

    if i >= 9:
        break

In [ ]:
# Read LIDAR data within the bounds of the image
matching_lasfiles = gdf_las[gdf_las.intersects(cellfile.geometry)].copy()
lidardata1 = []
lidardata2 = []
for lasfile in matching_lasfiles.itertuples():
    print(f"    - las file: {lasfile.path}")
    array1 = get_lasdata(
        lasfile.path, cellfile.xmin, cellfile.xmax, cellfile.ymin, cellfile.ymax
    )
    array1["lasfile"] = pathlib.Path(lasfile.path).stem
    array2 = get_lasground(
        lasfile.path, cellfile.xmin, cellfile.xmax, cellfile.ymin, cellfile.ymax
    )
    array2["lasfile"] = pathlib.Path(lasfile.path).stem
    lidardata1.append(array1)
    lidardata2.append(array2)
lidardata1 = pd.concat(lidardata1, ignore_index=True)
lidardata2 = pd.concat(lidardata2, ignore_index=True)
xstep = (cellfile.xmax - cellfile.xmin) / imgRGB.shape[1]
ystep = (cellfile.ymax - cellfile.ymin) / imgRGB.shape[0]
lidardata1["m"] = ((cellfile.ymax - lidardata1.Y) / ystep).astype(int)
lidardata1["n"] = ((lidardata1.X - cellfile.xmin) / xstep).astype(int)
lidardata2["m"] = ((cellfile.ymax - lidardata2.Y) / ystep).astype(int)
lidardata2["n"] = ((lidardata2.X - cellfile.xmin) / xstep).astype(int)

pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(lidardata1[["X", "Y", "Z"]].to_numpy())
plane_model, inliers = pcd.segment_plane(
    distance_threshold=0.01, ransac_n=3, num_iterations=1000
)
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

lidardata3 = pd.DataFrame(np.asarray(outlier_cloud.points), columns=["X", "Y", "Z"])
lidardata3["m"] = ((cellfile.ymax - lidardata3.Y) / ystep).astype(int)
lidardata3["n"] = ((lidardata3.X - cellfile.xmin) / xstep).astype(int)

In [ ]:
"""
  points_per_side (int or None): The number of points to be sampled
    along one side of the image. The total number of points is
    points_per_side**2. If None, 'point_grids' must provide explicit
    point sampling.
  points_per_batch (int): Sets the number of points run simultaneously
    by the model. Higher numbers may be faster but use more GPU memory.
  pred_iou_thresh (float): A filtering threshold in [0,1], using the
    model's predicted mask quality.
  stability_score_thresh (float): A filtering threshold in [0,1], using
    the stability of the mask under changes to the cutoff used to binarize
    the model's mask predictions.
  stability_score_offset (float): The amount to shift the cutoff when
    calculated the stability score.
  box_nms_thresh (float): The box IoU cutoff used by non-maximal
    suppression to filter duplicate masks.
  crop_n_layers (int): If >0, mask prediction will be run again on
    crops of the image. Sets the number of layers to run, where each
    layer has 2**i_layer number of image crops.
  crop_nms_thresh (float): The box IoU cutoff used by non-maximal
    suppression to filter duplicate masks between different crops.
  crop_overlap_ratio (float): Sets the degree to which crops overlap.
    In the first crop layer, crops will overlap by this fraction of
    the image length. Later layers with more crops scale down this overlap.
  crop_n_points_downscale_factor (int): The number of points-per-side
    sampled in layer n is scaled down by crop_n_points_downscale_factor**n.
  point_grids (list(np.ndarray) or None): A list over explicit grids
    of points used for sampling, normalized to [0,1]. The nth grid in the
    list is used in the nth crop layer. Exclusive with points_per_side.
  min_mask_region_area (int): If >0, postprocessing will be applied
    to remove disconnected regions and holes in masks with area smaller
    than min_mask_region_area. Requires opencv.
  output_mode (str): The form masks are returned in. Can be 'binary_mask',
    'uncompressed_rle', or 'coco_rle'. 'coco_rle' requires pycocotools.
    For large resolutions, 'binary_mask' may consume large amounts of
    memory.
    points_per_side: Optional[int] = 32,
    points_per_batch: int = 64,
    pred_iou_thresh: float = 0.88,
    stability_score_thresh: float = 0.95,
    stability_score_offset: float = 1.0,
    box_nms_thresh: float = 0.7,
    crop_n_layers: int = 0,
    crop_nms_thresh: float = 0.7,
    crop_overlap_ratio: float = 512 / 1500,
    crop_n_points_downscale_factor: int = 1,
    point_grids: Optional[List[np.ndarray]] = None,
    min_mask_region_area: int = 0,
"""

try:
    del sam
except NameError:
    pass

import torch

torch.cuda.empty_cache()

hd = True
device = "cpu"

if hd:
    from segment_anything_hq import SamAutomaticMaskGenerator, sam_model_registry

    sam = sam_model_registry["vit_h"](
        checkpoint="data/sam_weights/sam_hq_vit_h.pth"
    )
    sam.to(device=device)
else:
    from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

    sam = sam_model_registry["vit_h"](
        checkpoint="data/sam_weights/sam_vit_h_4b8939.pth"
    )
    sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    sam,
    points_per_batch=32,  # nodig om geen OoM op de GPU te krijgen
)

plt.close("all")
for imgId in imgIds:
    # COCO format img en annotation
    img = coco.loadImgs(imgId)[0]

    # Laad image in
    imgPath = imgDir.joinpath(img["file_name"])
    imgRGB = skimage.io.imread(imgPath)
    imgGray = skimage.io.imread(imgPath, as_gray=True)

    # Laad annotation in
    annIds = coco.getAnnIds(imgIds=img["id"], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)

    # SAM mask
    masks = mask_generator.generate(imgRGB)

    # gesegmenteerd
    plt.figure(figsize=(10, 10))
    plt.axis("off")
    plt.imshow(imgGray, cmap="gray")
    coco.showAnns(anns, draw_bbox=True)
    plt.tight_layout()
    plt.show()

    # SAM
    plt.figure(figsize=(10, 10))
    plt.axis("off")
    plt.imshow(imgGray, cmap="gray")
    show_anns(masks)
    plt.tight_layout()
    plt.show()

In [ ]:
voorbeeld_path = pathlib.Path(
    "276114_20230714 MUG Hoorn Enkhuizen orthomosaic deel 1/2023-12-11 10_00_58.048/images"
)

df_files = dict(name=[], img_path=[])
for p in voorbeeld_path.iterdir():
    if p.suffix == ".json" or p.suffix == ".jpeg":
        if p.stem not in df_files["name"]:
            df_files["name"].append(p.stem)
        #         if p.suffix == ".json":
        #             df_files["segment_path"].append(str(p))
        #         else:
        df_files["img_path"].append(str(p))
    else:
        print(f"unknown filetype {p.suffix=}, {p}")
df_files = pd.DataFrame(df_files)
display(df_files)

In [ ]:
plt.close("all")

for i, row in enumerate(df_files.itertuples()):
    # Find tile bounds (X, Y) based on name
    tile1a = row.name.split("_")[0]
    tile1b = int(row.name.split("_")[-1])
    cellfile = df_tilebounds[
        (df_tilebounds.index == tile1b) & (df_tilebounds.name == tile1a)
    ].copy()
    assert len(cellfile) == 1
    cellfile = cellfile.iloc[0, :].copy()
    print(f"  - {i+1:>02d}/{len(df_files):>02d} rgb tile: '{tile1a}' ('{tile1b}')")

    # Read image data
    imgdata = matplotlib.image.imread(row.img_path)

    # Read LIDAR data within the bounds of the image
    matching_lasfiles = df_las[df_las.intersects(cellfile.geometry)].copy()
    lidardata = []
    for lasfile in matching_lasfiles.itertuples():
        print(f"    - las file: {lasfile.path}")
        array = get_lasdata(
            lasfile.path, cellfile.xmin, cellfile.xmax, cellfile.ymin, cellfile.ymax
        )
        array["lasfile"] = pathlib.Path(lasfile.path).stem
        lidardata.append(array)
    lidardata = pd.concat(lidardata, ignore_index=True)
    xstep = (cellfile.xmax - cellfile.xmin) / imgdata.shape[1]
    ystep = (cellfile.ymax - cellfile.ymin) / imgdata.shape[0]
    lidardata["m"] = ((cellfile.ymax - lidardata.Y) / ystep).astype(int)
    lidardata["n"] = ((lidardata.X - cellfile.xmin) / xstep).astype(int)

    # Read segmentation data
    lidardata_obj = []
    with open(row.segment_path) as f:
        # Read json data
        jsondata = json.load(f)

        # Read the bitmap masked data in the same shape as the imgdata
        full_mask = {
            "opschot": np.zeros((imgdata.shape[0], imgdata.shape[1]), dtype=bool),
            "gras steenbekleding": np.zeros(
                (imgdata.shape[0], imgdata.shape[1]), dtype=bool
            ),
            "twijfel opschot": np.zeros(
                (imgdata.shape[0], imgdata.shape[1]), dtype=bool
            ),
        }
        mask_bounds = []
        for obj in jsondata["annotation"]["objects"]:
            cat_title = obj["classTitle"]

            # Update complete mask
            bool_mask = base64_2_mask(obj["bitmap"]["data"])
            n1, m1 = obj["bitmap"]["origin"]
            m2 = m1 + bool_mask.shape[0]
            n2 = n1 + bool_mask.shape[1]
            full_mask[cat_title][m1:m2, n1:n2] = bool_mask

            # mask bounds
            bbox = shapely.box(n1, m1, n2, m2).boundary
            mask_bounds.append((cat_title, bbox))

            # relevant lidardata
            sublidar = lidardata[
                (lidardata.m >= m1)
                & (lidardata.m < m2)
                & (lidardata.n >= n1)
                & (lidardata.n < n2)
            ].copy()
            lidardata_obj.append(sublidar)

        mask_bounds = gpd.GeoDataFrame(
            pd.DataFrame(mask_bounds, columns=["mask_type", "geometry"]),
            geometry="geometry",
        )

    cat_int = {
        "opschot": 1,
        "gras steenbekleding": 2,
        "twijfel opschot": 3,
    }
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 8), dpi=100)

    # Show bounding boxes over image
    axs[0, 0].pcolormesh(imgdata)
    mask_bounds.plot(ax=axs[0, 0], column="mask_type")
    axs[0, 0].invert_yaxis()

    # Show only the masked data with bounding boxes
    mask_bounds.plot(
        ax=axs[0, 1],
        column="mask_type",
        legend=True,
        legend_kwds={"loc": "center left"},
    )
    for obj_title, arrmask in full_mask.items():
        if not arrmask.flatten().any():
            continue
        arrmask = np.tile(arrmask[:, :, np.newaxis], (1, 1, 3))
        arrmask = np.ma.masked_array(imgdata, ~arrmask)
        axs[0, 1].pcolormesh(arrmask)
    axs[0, 1].invert_yaxis()
    axs[0, 1].set_xlim(axs[0, 0].get_xlim())
    axs[0, 1].set_ylim(axs[0, 0].get_ylim())
    leg = axs[0, 1].get_legend()
    leg.set_bbox_to_anchor((1, 0.5))

    # Show lidar data with boxes
    mask_bounds.plot(ax=axs[0, 1], column="mask_type")
    lidardata_plt = gpd.GeoDataFrame(
        lidardata, geometry=gpd.GeoSeries.from_xy(lidardata.n, lidardata.m)
    )
    lidardata_plt.plot(ax=axs[1, 0], column="Z")
    axs[1, 0].invert_yaxis()
    axs[1, 0].set_xlim(axs[0, 0].get_xlim())
    axs[1, 0].set_ylim(axs[0, 0].get_ylim())

    mask_bounds.plot(
        ax=axs[1, 1],
        column="mask_type",
        legend=True,
        legend_kwds={"loc": "center left"},
    )
    for lidarsubset in lidardata_obj:
        lidardata_plt = gpd.GeoDataFrame(
            lidarsubset, geometry=gpd.GeoSeries.from_xy(lidarsubset.n, lidarsubset.m)
        )
        lidardata_plt.plot(ax=axs[1, 1], column="Z")
    axs[1, 1].invert_yaxis()
    axs[1, 1].set_xlim(axs[0, 0].get_xlim())
    axs[1, 1].set_ylim(axs[0, 0].get_ylim())
    leg = axs[1, 1].get_legend()
    leg.set_bbox_to_anchor((1, 0.5))

    fig.tight_layout()